In [86]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# from linearmodels.panel.data import PanelData
# from linearmodels.panel import PanelOLS, PooledOLS, RandomEffects, compare
import matplotlib.pyplot as plt
import pyblp

In [87]:
product_data=pd.read_csv("../data/merged/len4_ndb_agg_blp_DropNever.csv",encoding="shift-jis",index_col=0)
product_data["market_ids"]=product_data["id"].astype(int).astype(str)+"-"+product_data["year"].astype(str)
product_data.rename(columns={"薬価":"prices","r_share":"shares",
                    "elasped_0":"demand_instruments0",
                    "elasped_1":"demand_instruments1",
                    'elasped_2': 'demand_instruments2',
                    # 'elasped_3': 'demand_instruments3',
                    # 'elasped_4': 'demand_instruments4',
                    # 'elasped_5': 'demand_instruments5',
                    # 'elasped_6': 'demand_instruments6',
                    # 'elasped_7': 'demand_instruments7',
                    # 'elasped_8': 'demand_instruments8',
                    # 'elasped_9': 'demand_instruments9'
                    "year":"demand_instruments3",
                    "generic_per":"demand_instruments4",
                    },inplace=True)
product_data=product_data.loc[product_data["shares"]>0]
# data["shares"]=data["r_share"]
product_data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,prices,後発品区分,総計,demand_instruments3,in_hospital,...,id_3992.0,id_3999.0,id_4413.0,id_4490.0,id_6250.0,id_6290.0,id_7990.0,q_share,shares,market_ids
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,0,0,0,0,0,0,0,0.182126,0.182126,1145-2014
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,0,0,0,0,0,0,0,0.080302,0.080302,1145-2014
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,0,0,0,0,0,0,0,0.066275,0.066275,1145-2014
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,0,0,0,0,0,0,0,0.410299,0.410299,1145-2014
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,0,0,0,0,0,0,0,0.260997,0.260997,1145-2014


In [88]:
logit_formulation = pyblp.Formulation('prices+oral+in_hospital+後発品区分', absorb='C(id)')
logit_formulation

prices + oral + in_hospital + 後発品区分 + Absorb[C(id)]

In [89]:
product_data["shares"]-=1e-10
problem = pyblp.Problem(logit_formulation, product_data)
problem

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T     N      K1    MD    ED 
---  ------  ----  ----  ----
344  104614   4     7     1  

Formulations:
     Column Indices:          0      1         2         3  
--------------------------  ------  ----  -----------  -----
X1: Linear Characteristics  prices  oral  in_hospital  後発品区分


Dimensions:
 T     N      K1    MD    ED 
---  ------  ----  ----  ----
344  104614   4     7     1  

Formulations:
     Column Indices:          0      1         2         3  
--------------------------  ------  ----  -----------  -----
X1: Linear Characteristics  prices  oral  in_hospital  後発品区分

In [90]:
logit_results = problem.solve()
logit_results

Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 1    +5.482191E+02     0      +1.021972E+02  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +7.810165E+00     0      +4.396221E+02      +2.029135E+07  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices            oral          in_hospital         後発品区分     
---------------  ---------------  ---------------  ---------------
 -7.386122E-03    +1.754471E+00    +3

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -------------  -------  ----------------  -----------------
 2    +7.810165E+00     0      +4.396221E+02      +2.029135E+07  

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
    prices            oral          in_hospital         後発品区分     
---------------  ---------------  ---------------  ---------------
 -7.386122E-03    +1.754471E+00    +3.227856E-01    -3.872838E+00 
(+3.108818E-03)  (+2.697630E-01)  (+7.528188E-02)  (+9.434999E-01)

## random coef

In [91]:
X1_formulation = pyblp.Formulation('prices+oral+in_hospital',absorb='C(id)',absorb_method="lsmr")
X2_formulation = pyblp.Formulation('-1+I(-prices)+後発品区分')
product_formulations = (X1_formulation, X2_formulation)
agent_formulation = pyblp.Formulation('1')
pr_integration = pyblp.Integration('product', size=5)
pr_integration


Configured to construct nodes and weights according to the level-5 Gauss-Hermite product rule with options {}.

In [93]:
mc_integration = pyblp.Integration('monte_carlo', size=5, specification_options={'seed': 0})
mc_integration
mc_problem = pyblp.Problem(product_formulations, product_data ,integration=mc_integration,rc_types=["log","linear"])
# mc_problem
# optim = pyblp.Optimization('nelder-mead',compute_gradient=False)
# optim = pyblp.Optimization('slsqp')
optim = pyblp.Optimization('bfgs',{'gtol': 1e-4})
results1 = mc_problem.solve(sigma=[[1,0],[0,1]], optimization=optim)
results1

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:02.

Dimensions:
 T     N      I     K1    K2    MD    ED 
---  ------  ----  ----  ----  ----  ----
344  104614  1720   3     2     6     1  

Formulations:
       Column Indices:            0       1         2     
-----------------------------  -------  -----  -----------
 X1: Linear Characteristics    prices   oral   in_hospital
X2: Nonlinear Characteristics  -prices  後発品区分             
Solving the problem ...

Nonlinear Coefficient Initial Values:
Types:  |  Sigma:      -prices         後発品区分    
------  |  -------  -------------  -------------
 Log    |  -prices  +1.000000E+00               
Linear  |   後発品区分   +0.000000E+00  +1.000000E+00

Nonlinear Coefficient Lower Bounds:
Types:  |  Sigma:      -prices         後発品区分    
------  |  -------  -------------  -------------
 Log    |  -prices      -INF                    
Linear  |   後発品区分   +0.000000E+00      -INF     

Nonlinea

KeyboardInterrupt: 

In [ ]:
print(results1)

Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  -------------  -------------  --------------  --------------  -------  ----------------  -----------------
 2    +3.575523E+00  +2.078739E+00  -3.525797E+09   +3.846079E+09      0      +2.292341E+01      +2.166782E+06  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 03:32:32       No           2            46        25812469     77468261  

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Sigma:      prices            後発品区分     
------  ---------------  ---------------
prices   -4.967954E-01                  
        (+

In [ ]:
# import sys
# filename = "./results/blp_0902.txt"
# with open(filename, "w") as file:
#     # Redirect the print output to the file
#     original_stdout = sys.stdout
#     sys.stdout = file
    
#     # Print the contents of the ProblemResults object
#     print(results1)
    
#     # Reset the print output back to the original stdout
#     sys.stdout = original_stdout

In [94]:
pr_integration = pyblp.Integration('product', size=5)
pr_problem = pyblp.Problem(product_formulations, product_data, integration=pr_integration)
optim = pyblp.Optimization('bfgs',{'gtol': 1e-4})
results2 = pr_problem.solve(sigma=np.ones((2,2)), optimization=optim)
results2


Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:02.

Dimensions:
 T     N      I     K1    K2    MD    ED 
---  ------  ----  ----  ----  ----  ----
344  104614  8600   3     2     6     1  

Formulations:
       Column Indices:            0       1         2     
-----------------------------  -------  -----  -----------
 X1: Linear Characteristics    prices   oral   in_hospital
X2: Nonlinear Characteristics  -prices  後発品区分             
Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:      -prices         後発品区分      |  Sigma Squared:     -prices         後発品区分    
-------  -------------  -------------  |  --------------  -------------  -------------
-prices  +1.000000E+00                 |     -prices      +1.000000E+00  +1.000000E+00
 後発品区分   +1.000000E+00  +1.000000E+00  |      後発品区分       +1.000000E+00  +2.000000E+00
Starting optimization ...


At least one error was encountered. As long as the optimizatio

KeyboardInterrupt: 